### Names : Alice Devilder, Kanupriya Jain

## **Steepest descent method**

In this program, we will be implementing the steepest descent method to minimize $\textit{Rosenbrock's function}$
$$
f: \mathbb{R}^2 \rightarrow \mathbb{R}, \hspace{0.5cm} f(x_1, x_2) = (1 - x_1)^2 + 100(x_2 - x_1^2)^2
$$

and $\textit{Himmelblau's function}$.
$$
g: \mathbb{R}^2 \rightarrow \mathbb{R}, \hspace{0.5cm} g(x_1, x_2) = (x_2^2 + x_2 - 11)^2 + (x_1 + x_2^2 - 7)^2
$$

with the starting point $x^0 = (0,0)^T$ and then $\~{x}^0 = (\pi + 1, \pi - 1)^T$ (for each function).

In [1]:
import numpy as np
import random as rd

from scipy.optimize import line_search
import plotly.graph_objects as go

### Rosenbrock's and Himmelblau's function and their gradient

In [2]:
def rosenbrock(x):
    return (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2

def gradient_rosenbrock(x):
    return np.array([-2*(1 - x[0]) - 400*x[0]*(x[1] - x[0]**2),
                     200*(x[1] - x[0]**2)])

def himmelblau(x):
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2

def gradient_himmelblau(x):
    return np.array([4*x[0]*(x[0]**2 + x[1] - 11) + 2*(x[0] + x[1]**2 - 7),
                     2*(x[0]**2 + x[1] - 11) + 4*x[1]*(x[0] + x[1]**2 - 7)])

### Plot of the Rosenbrock function (interactive)

In [3]:
# Generate data for plotting
x_values = np.linspace(-2, 2, 100)
y_values = np.linspace(-1, 3, 100)
X, Y = np.meshgrid(x_values, y_values)
Z = rosenbrock([X, Y])

# Create 3D surface plot using Plotly
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

# Mark the minimum point on the plot
min_point = np.array([1, 1, rosenbrock([1, 1])])
fig.add_trace(go.Scatter3d(x=[min_point[0]], y=[min_point[1]], z=[min_point[2]],
                           mode='markers', marker=dict(color='red', size=5),
                           name='Minimum'))

# Set layout for better visibility
fig.update_layout(title="3D plot of the Rosenbrock's function - overview",
                  scene=dict(zaxis=dict(range=[0, 1000]),
                             camera=dict(up=dict(x=0, y=0, z=1),
                                         center=dict(x=0, y=0, z=0),
                                         eye=dict(x=1.25, y=1.25, z=1.25))),
                  width=800, height=600, scene_aspectmode='cube')

# Show the interactive plot
fig.show()

In [4]:
# Generate data for plotting
x_values = np.linspace(-2, 2, 100)
y_values = np.linspace(-1, 3, 100)
X, Y = np.meshgrid(x_values, y_values)
Z = rosenbrock([X, Y])

# Create 3D surface plot using Plotly
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

# Mark the minimum point on the plot
min_point = np.array([1, 1, rosenbrock([1, 1])])
fig.add_trace(go.Scatter3d(x=[min_point[0]], y=[min_point[1]], z=[min_point[2]],
                           mode='markers', marker=dict(color='red', size=5),
                           name='Minimum'))

# Set layout for better visibility
fig.update_layout(title="3D plot of the Rosenbrock's function - zoom on the minimum",
                  scene=dict(zaxis=dict(range=[0, 5]),
                             camera=dict(up=dict(x=0, y=0, z=1),
                                         center=dict(x=0, y=0, z=0),
                                         eye=dict(x=1.25, y=1.25, z=1.25))),
                  width=800, height=600, scene_aspectmode='cube')

# Show the interactive plot
fig.show()

On this graph, we can see that the (unique) minimum of the Rosenbrock's function should be 0 and it is obtained at the point (1,1).

### Plot of the Himmelblau's function (interative)

In [5]:
import plotly.graph_objects as go

# Generate data for plotting
x_values = np.linspace(-4, 4, 100)
y_values = np.linspace(-4, 4, 100)
X, Y = np.meshgrid(x_values, y_values)
Z = himmelblau([X, Y])

# Create 3D surface plot using Plotly
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

# Mark the minimum point on the plot
min_point = np.array([[3, 2, himmelblau([3, 2])],
                      [-2.805118, 3.131312, himmelblau([-2.805118, 3.131312])],
                      [-3.779310, -3.283186, himmelblau([-3.779310, -3.283186])],
                      [3.584458, -1.848126, himmelblau([3.584458, -1.848126])]])
fig.add_trace(go.Scatter3d(x=[min_point[0][0], min_point[1][0], min_point[2][0], min_point[3][0]],
                           y=[min_point[0][1], min_point[1][1], min_point[2][1], min_point[3][1]],
                           z=[min_point[0][2], min_point[1][2], min_point[2][2], min_point[3][2]],
                           mode='markers', marker=dict(color='red', size=5),
                           name='Minimum'))

# Set layout for better visibility
fig.update_layout(title="3D plot of the Himmelblau's function with 4 minima",
                  scene=dict(zaxis=dict(range=[0, 200]),
                             camera=dict(up=dict(x=0, y=0, z=1),
                                         center=dict(x=0, y=0, z=0),
                                         eye=dict(x=1.25, y=1.25, z=1.25))),
                  width=800, height=600, scene_aspectmode='cube')

# Show the interactive plot
fig.show()

On this graph, we can see that there are 4 minima for this function.

## 1. The steepest descent algorithm with a constant stepzise

Let's first determine the best stepsize. To this end, we will implement the Goldstein-Armijo and the Wolfe-Powell algorithms.

In [ ]:
def goldstein_armijo_stepsize(x, f, grad, sigma=1e-3, beta1=1e-3, beta2=0.8, max_iterations=10000):
    delta = rd.random()
    d= -grad(x)
    stepsize = (-sigma * np.dot(grad(x), d)) / (np.linalg.norm(d)**2)
    iteration = 0

    while iteration < max_iterations :
        if f(x + stepsize * d) <= f(x) + delta * stepsize * np.dot(grad(x), d):
            stepsize = stepsize
            break
        else:
            stepsize = rd.uniform(beta1*stepsize, beta2*stepsize)
            iteration += 1

    if iteration == max_iterations:
        print("Maximum number of iterations reached")
    return stepsize

In [ ]:
def wolfe_powell_stepsize(x, f, grad, delta=1e-4, beta=0.8, max_iterations=10000):
    stepsize = 1.0  # Initial step size
    d = -grad(x) # Initial search direction
    iteration = 0

    while iteration < max_iterations:
        f_x = f(x)
        grad_x = grad(x)
        x_next = x + stepsize * d

        # Armijo condition
        armijo_condition = f(x_next) <= f_x + delta * stepsize * np.dot(grad_x, d)

        # Curvature condition
        curvature_condition = np.dot(grad(x_next), d) >= beta * np.dot(grad_x, d)

        if armijo_condition and curvature_condition:
            break

        # Adjust the step size using line search
        result = line_search(f, grad,x, d)
        if not result[0]:
            break

        stepsize = result[0]
        iteration += 1

    if iteration == max_iterations:
        print("Maximum number of iterations reached")
    return stepsize


Let's now implement the steepest descent algorithm with a constant stepsize.

In [ ]:
def steepest_descent_constant(x0, grad, stepsize, epsilon, max_iterations):
    x = x0
    iteration = 0

    # Loop until the gradient is close to zero or the maximum number of iterations is reached
    while np.linalg.norm(grad(x)) > epsilon and iteration < max_iterations:
        p = -grad(x)
        x_next = x + stepsize * p

        # Another stopping criterion
        if np.linalg.norm(x_next - x) < epsilon:
            break

        x = x_next
        iteration += 1

    if iteration == max_iterations:
        print("Maximum number of iterations reached")

    print("Number of total iterations: ", iteration)
    return x

We want compute the minimum of the rosenbrock's and himmelblau's functions using the steepest descent method with
* Constant stepsize chosen randomly : 0.001
* 10
* 1e-6
* Constant stepsize using Wolfe-Powell algorithm
* Constant stepsize using Goldstein-Armijo algorithm (normally better)

with the $x_0$ and $\~{x}^0$.

In [ ]:
# Initial values
x0 = np.array([0, 0])
x_tilda = np.array([np.pi + 1, np.pi - 1])

# Parameters
sigma = 1e-3
beta1 = 1e-3
beta2 = 0.8

epsilon = 1e-5
max_iterations = 10000

# Different step sizes
stepsize1 = 0.001
stepsize2 = 10
stepsize3 = 1e-6

stepsize_ga = goldstein_armijo_stepsize(x0, rosenbrock, gradient_rosenbrock)
print("Stepsize using Goldstein-armijo algorithm: ", stepsize_ga)
stepsize_wp = wolfe_powell_stepsize(x0, rosenbrock, gradient_rosenbrock)
print("Stepsize using Wolfe-Powell algorithm: ", stepsize_wp)

stepsize_ga_tilda = goldstein_armijo_stepsize(x_tilda, rosenbrock, gradient_rosenbrock)
print("Stepsize using Goldstein-armijo algorithm: ", stepsize_ga_tilda)
stepsize_wp_tilda = wolfe_powell_stepsize(x_tilda, rosenbrock, gradient_rosenbrock)
print("Stepsize using Wolfe-Powell algorithm: ", stepsize_wp_tilda)

stepsize_ga_himmel = goldstein_armijo_stepsize(x0, himmelblau, gradient_himmelblau)
print("Stepsize using Goldstein-armijo algorithm: ", stepsize_ga_himmel)
stepsize_wp_himmel = wolfe_powell_stepsize(x0, himmelblau, gradient_himmelblau)
print("Stepsize using Wolfe-Powell algorithm: ", stepsize_wp_himmel)

stepsize_ga_himmel_tilda = goldstein_armijo_stepsize(x_tilda, himmelblau, gradient_himmelblau)
print("Stepsize using Goldstein-armijo algorithm: ", stepsize_ga_himmel_tilda)
stepsize_wp_himmel_tilda = wolfe_powell_stepsize(x_tilda, himmelblau, gradient_himmelblau)
print("Stepsize using Wolfe-Powell algorithm: ", stepsize_wp_himmel_tilda)

# Stepsizes
stepsizes_rosen_x0 = [stepsize1, stepsize2, stepsize3, stepsize_ga, stepsize_wp]
stepsizes_rosen_x_tilda = [stepsize1, stepsize2, stepsize3, stepsize_ga_tilda, stepsize_wp_tilda]

stepsizes_himmel_x0 = [stepsize1, stepsize2, stepsize3, stepsize_ga_himmel, stepsize_wp_himmel]
stepsizes_himmel_x_tilda = [stepsize1, stepsize2, stepsize3, stepsize_ga_himmel_tilda, stepsize_wp_himmel_tilda]

Stepsize using Goldstein-armijo algorithm:  0.001
Stepsize using Wolfe-Powell algorithm:  0.08677323593467849
Stepsize using Goldstein-armijo algorithm:  9.777062645715805e-06
Stepsize using Wolfe-Powell algorithm:  0.00032862843885677636
Stepsize using Goldstein-armijo algorithm:  0.0009999999999999998
Stepsize using Wolfe-Powell algorithm:  0.13334137278472633
Stepsize using Goldstein-armijo algorithm:  0.0009999999999999998
Stepsize using Wolfe-Powell algorithm:  0.0186448807584061


In [ ]:
print("Rosenbrock's function with x0 \n")
for stepsize in stepsizes_rosen_x0:

    minimizer = steepest_descent_constant(x0, gradient_rosenbrock, stepsize, epsilon, max_iterations)
    minimized_value = rosenbrock(minimizer)

    print("Minimizer with stepsize", stepsize,": ", minimizer)
    print("Minimized Value:", minimized_value, "\n")

Rosenbrock's function with x0 

Number of total iterations:  8313
Minimizer with stepsize 0.001 :  [0.98891964 0.97791742]
Minimized Value: 0.0001229735658279806 



C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:5: RuntimeWarning:

overflow encountered in scalar multiply

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:6: RuntimeWarning:

overflow encountered in scalar multiply

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:5: RuntimeWarning:

overflow encountered in scalar subtract

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:6: RuntimeWarning:

overflow encountered in scalar subtract



Maximum number of iterations reached
Number of total iterations:  10000
Minimizer with stepsize 10 :  [-931264064  291641344]
Minimized Value: -513606527 

Number of total iterations:  0
Minimizer with stepsize 1e-06 :  [0 0]
Minimized Value: 1 

Number of total iterations:  8313
Minimizer with stepsize 0.001 :  [0.98891964 0.97791742]
Minimized Value: 0.0001229735658279806 

Number of total iterations:  9
Minimizer with stepsize 0.08677323593467849 :  [nan inf]
Minimized Value: nan 



C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:2: RuntimeWarning:

overflow encountered in scalar multiply

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\277637736.py:8: RuntimeWarning:

invalid value encountered in add



In [ ]:
print("Rosenbrock's function with x_tilda \n")
for stepsize in stepsizes_rosen_x_tilda:

    minimizer = steepest_descent_constant(x_tilda, gradient_rosenbrock, stepsize, epsilon, max_iterations)
    minimized_value = rosenbrock(minimizer)

    print("Minimizer with stepsize", stepsize,": ", minimizer)
    print("Minimized Value:", minimized_value, "\n")

Rosenbrock's function with x_tilda 

Number of total iterations:  7
Minimizer with stepsize 0.001 :  [-inf  inf]
Minimized Value: nan 

Number of total iterations:  5
Minimizer with stepsize 10 :  [-inf  inf]
Minimized Value: nan 



C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:5: RuntimeWarning:

overflow encountered in scalar power

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:6: RuntimeWarning:

overflow encountered in scalar power

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:5: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:6: RuntimeWarning:

invalid value encountered in scalar subtract

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:2: RuntimeWarning:

invalid value encountered in scalar subtract



Number of total iterations:  2249
Minimizer with stepsize 1e-06 :  [1.61943175 2.60959641]
Minimized Value: 0.4004990602835588 

Number of total iterations:  504
Minimizer with stepsize 5.993099594503182e-06 :  [1.61513543 2.60802209]
Minimized Value: 0.37843260954999425 

Maximum number of iterations reached
Number of total iterations:  10000
Minimizer with stepsize 0.00032862843885677636 :  [1.62015883 2.62666219]
Minimized Value: 0.38490236979982617 



In [ ]:
print("Himmelblau's function with x0 \n")
for stepsize in stepsizes_himmel_x0:

    minimizer = steepest_descent_constant(x0, gradient_himmelblau, stepsize, epsilon, max_iterations)
    minimized_value = himmelblau(minimizer)

    print("Minimizer with stepsize", stepsize,": ", minimizer)
    print("Minimized Value:", minimized_value, "\n")

Himmelblau's function with x0 

Number of total iterations:  319
Minimizer with stepsize 0.001 :  [2.99985444 2.00035126]
Minimized Value: 1.8591962748049284e-06 



C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:12: RuntimeWarning:

overflow encountered in scalar multiply

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:13: RuntimeWarning:

overflow encountered in scalar multiply

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:12: RuntimeWarning:

overflow encountered in scalar add

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:13: RuntimeWarning:

overflow encountered in scalar add



Maximum number of iterations reached
Number of total iterations:  10000
Minimizer with stepsize 10 :  [-974742208 1529885760]
Minimized Value: -367179350 



C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:9: RuntimeWarning:

overflow encountered in scalar add



Maximum number of iterations reached
Number of total iterations:  10000
Minimizer with stepsize 1e-06 :  [0.17290899 0.25033464]
Minimized Value: 160.67084908439287 

Number of total iterations:  319
Minimizer with stepsize 0.0009999999999999998 :  [2.99985444 2.00035126]
Minimized Value: 1.8591962748049284e-06 

Number of total iterations:  8
Minimizer with stepsize 0.13334137278472633 :  [             inf -5.92457345e+248]
Minimized Value: inf 



C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:12: RuntimeWarning:

overflow encountered in scalar power

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:13: RuntimeWarning:

overflow encountered in scalar power

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:13: RuntimeWarning:

invalid value encountered in scalar add

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:9: RuntimeWarning:

overflow encountered in scalar power



In [ ]:
print("Himmelblau's function with x_tilda \n")
for stepsize in stepsizes_himmel_x_tilda:

    minimizer = steepest_descent_constant(x_tilda, gradient_himmelblau, stepsize, epsilon, max_iterations)
    minimized_value = himmelblau(minimizer)

    print("Minimizer with stepsize", stepsize,": ", minimizer)
    print("Minimized Value:", minimized_value, "\n")

Himmelblau's function with x_tilda 

Number of total iterations:  244
Minimizer with stepsize 0.001 :  [3.00014674 1.99964557]
Minimized Value: 1.8917551049319542e-06 

Number of total iterations:  5
Minimizer with stepsize 10 :  [            -inf -3.29754496e+265]
Minimized Value: nan 



C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:12: RuntimeWarning:

overflow encountered in scalar multiply

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:12: RuntimeWarning:

overflow encountered in scalar power

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:12: RuntimeWarning:

invalid value encountered in scalar add

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:13: RuntimeWarning:

overflow encountered in scalar power

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:13: RuntimeWarning:

invalid value encountered in scalar add

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:9: RuntimeWarning:

overflow encountered in scalar power

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:9: RuntimeWarning:

invalid value encountered in scalar add



Maximum number of iterations reached
Number of total iterations:  10000
Minimizer with stepsize 1e-06 :  [3.410996   1.97595442]
Minimized Value: 6.915999834023553 

Number of total iterations:  244
Minimizer with stepsize 0.0009999999999999998 :  [3.00014674 1.99964557]
Minimized Value: 1.8917551049319542e-06 

Number of total iterations:  18
Minimizer with stepsize 0.0186448807584061 :  [2.99999101 2.00000753]
Minimized Value: 2.6018558095220125e-09 



The admissible errors for this result are defined by the value of `epsilon`, which is set to `1e-5` in the code. This means that the algorithm will continue iterating until the norm of the gradient is less than `1e-5`, indicating that the algorithm has converged to a minimum.

## 2. The steepest descent algorithm with variable stepzise

In [ ]:
def steepest_descent_variable(x0, f, grad, stepsize_algo, epsilon, max_iterations):
    x = x0
    iteration = 0

    # Loop until the gradient is close to zero or the maximum number of iterations is reached
    while np.linalg.norm(grad(x)) > epsilon and iteration < max_iterations:
        p = -grad(x)

        # Compute the step size using the stepsize algorithm (e.g. Goldstein-Armijo)
        stepsize = stepsize_algo(x, f, grad)
        x_next = x + stepsize * p

        # Another stopping criterion
        if np.linalg.norm(x_next - x) < epsilon:
            break

        x = x_next
        iteration += 1

    if iteration == max_iterations:
        print("Maximum number of iterations reached")

    print("Number of total iterations: ", iteration)
    return x

In [ ]:
# Initial values
x0 = np.array([0, 0])
x_tilda = np.array([np.pi + 1, np.pi - 1])

# Parameters
epsilon = 1e-5
max_iterations = 10000


In [ ]:
print("Rosenbrock's function with x0 \n")

for algo in [goldstein_armijo_stepsize, wolfe_powell_stepsize]:

    minimizer = steepest_descent_variable(x0, rosenbrock, gradient_rosenbrock, algo, epsilon, max_iterations)
    minimized_value = rosenbrock(minimizer)

    print("Minimizer using ", algo,": ", minimizer)
    print("Minimized Value:", minimized_value, "\n")

Rosenbrock's function with x0 

Number of total iterations:  756
Minimizer using  <function goldstein_armijo_stepsize at 0x00000215FF523370> :  [0.60810578 0.36786639]
Minimized Value: 0.1539521222623227 



Number of total iterations:  1725
Minimizer using  <function wolfe_powell_stepsize at 0x00000215FF5236D0> :  [0.9962254  0.99243299]
Minimized Value: 1.4350375236686046e-05 



In [ ]:
print("Rosenbrock's function with x_tilda \n")

for algo in [goldstein_armijo_stepsize, wolfe_powell_stepsize]:

    minimizer = steepest_descent_variable(x_tilda, rosenbrock, gradient_rosenbrock, algo, epsilon, max_iterations)
    minimized_value = rosenbrock(minimizer)

    print("Minimizer using ", algo,": ", minimizer)
    print("Minimized Value:", minimized_value, "\n")

Rosenbrock's function with x_tilda 

Number of total iterations:  17
Minimizer using  <function goldstein_armijo_stepsize at 0x00000215FF523370> :  [1.59086314 2.53187718]
Minimized Value: 0.3492256780732822 



c:\Users\alice\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_linesearch.py:314: LineSearchWarning:

The line search algorithm did not converge

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:2: RuntimeWarning:

overflow encountered in scalar power

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\432770692.py:5: RuntimeWarning:

overflow encountered in scalar multiply

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\2523835697.py:12: RuntimeWarning:

invalid value encountered in scalar add

c:\Users\alice\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_linesearch.py:284: RuntimeWarning:

invalid value encountered in multiply

C:\Users\alice\AppData\Local\Temp\ipykernel_53720\2523835697.py:9: RuntimeWarning:

invalid value encountered in add

c:\Users\alice\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_linesearch.py:284: RuntimeWarning:

invalid value encountered in add

c:\Users\alice\A

Maximum number of iterations reached
Number of total iterations:  10
Minimizer using  <function wolfe_powell_stepsize at 0x00000215FF5236D0> :  [nan inf]
Minimized Value: nan 



C:\Users\alice\AppData\Local\Temp\ipykernel_53720\1858849636.py:11: RuntimeWarning:

invalid value encountered in add



In [ ]:
print("Himmelblau's function with x0 \n")

for algo in [goldstein_armijo_stepsize, wolfe_powell_stepsize]:

    minimizer = steepest_descent_variable(x0, himmelblau, gradient_himmelblau, algo, epsilon, max_iterations)
    minimized_value = himmelblau(minimizer)

    print("Minimizer using ", algo,": ", minimizer)
    print("Minimized Value:", minimized_value, "\n")

Himmelblau's function with x0 

Number of total iterations:  320
Minimizer using  <function goldstein_armijo_stepsize at 0x00000215FF523370> :  [2.99985144 2.00035849]
Minimized Value: 1.9365109064573575e-06 

Number of total iterations:  13
Minimizer using  <function wolfe_powell_stepsize at 0x00000215FF5236D0> :  [2.99999627 1.9999943 ]
Minimized Value: 1.4922788926435106e-09 



In [ ]:
print("Himmelblau's function with x_tilda \n")

for algo in [goldstein_armijo_stepsize, wolfe_powell_stepsize]:

    minimizer = steepest_descent_variable(x_tilda, himmelblau, gradient_himmelblau, algo, epsilon, max_iterations)
    minimized_value = himmelblau(minimizer)

    print("Minimizer using ", algo,": ", minimizer)
    print("Minimized Value:", minimized_value, "\n")

Himmelblau's function with x_tilda 

Number of total iterations:  190
Minimizer using  <function goldstein_armijo_stepsize at 0x00000215FF523370> :  [3.00064271 1.99844562]
Minimized Value: 3.635214462848667e-05 

Number of total iterations:  10
Minimizer using  <function wolfe_powell_stepsize at 0x00000215FF5236D0> :  [2.99999808 2.00000479]
Minimized Value: 3.429390719875568e-10 

